In [1]:
import os
import json
import asyncio
import aiohttp
import time
from typing import Dict, Any, Optional, List
from dataclasses import dataclass
import logging
import sys

# from .llm_agents import BaseLLMAgent, LLMConfig, ModelType

In [2]:
os.system('source ~/.bashrc')

0

In [ ]:
OPENROUTER_API_KEY="redacted"

In [14]:
# OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
url = f"https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://github.com/negotiation-research",
    "X-Title": "Negotiation Research"
}

# connector = aiohttp.TCPConnector(force_close=True)
# session = aiohttp.ClientSession(
#     connector=connector,
#     json_serialize=lambda x: json.dumps(x, ensure_ascii=False)
# )

In [5]:
messages = [
    {"role": "user", "content": "Hello, how are you? What is your name?"}
]

payload = {
    "model": "deepseek/deepseek-r1-0528",
    "messages": messages,
    "temperature": 0.7,
    "max_tokens": 4000,  # Cap at 4000 for safety
}

In [6]:
timeout=30.0

In [11]:
async def prompt_openrouter(url, headers, payload, timeout) -> str:
    connector = aiohttp.TCPConnector(force_close=True)
    session = aiohttp.ClientSession(
        connector=connector,
        json_serialize=lambda x: json.dumps(x, ensure_ascii=False)
    )

    response = await session.post(
        url,
        headers=headers,
        json=payload,
        timeout=aiohttp.ClientTimeout(total=timeout)
    )
    data = await response.json(encoding='utf-8')
    print(data)
    return data["choices"][0]["message"]["content"]

In [15]:
content = await prompt_openrouter(url, headers, payload, timeout)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x146cce310ec0>


{'id': 'gen-1768951873-YvoQXDfP30M0WcHAwPsN', 'provider': 'Nebius', 'model': 'deepseek/deepseek-r1-0528', 'object': 'chat.completion', 'created': 1768951873, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': "Hello! 😊 I'm doing great — just running smoothly and ready to help!  \nMy name is **DeepSeek-R1**, but you can call me **DeepSeek** for short. What would you like me to call you?  \nAnd how about you — how's your day going? Anything on your mind? 💬", 'refusal': None, 'reasoning': 'Okay, the user started with a friendly greeting: "Hello, how are you? What is your name?" Pretty standard opener, but nice and polite. \n\nHmm, they\'re asking two things at once - my status and my identity. That\'s common in human conversations but interesting for an AI interaction. They might be testing if I can handle multiple questions smoothly, or just being casually conversational. \n\nFirst part: "How are

In [16]:
content

"Hello! 😊 I'm doing great — just running smoothly and ready to help!  \nMy name is **DeepSeek-R1**, but you can call me **DeepSeek** for short. What would you like me to call you?  \nAnd how about you — how's your day going? Anything on your mind? 💬"

In [14]:
url

'https://openrouter.ai/api/v1/chat/completions'

In [ ]:
headers

In [16]:
payload

{'model': 'deepseek/deepseek-r1-0528',
 'messages': [{'role': 'user',
   'content': 'Hello, how are you? What is your name?'}],
 'temperature': 0.7,
 'max_tokens': 4000}

In [22]:
timeout

30.0

In [9]:
connector = aiohttp.TCPConnector(force_close=True)
session = aiohttp.ClientSession(
    connector=connector,
    json_serialize=lambda x: json.dumps(x, ensure_ascii=False)
)

In [ ]:
await session.post(
    url,
    headers=headers,
    json=payload,
    timeout=aiohttp.ClientTimeout(total=30.0)
)

In [ ]:
import json
import aiohttp

async def prompt_openrouter(url, headers, payload, timeout) -> str:
    connector = aiohttp.TCPConnector(force_close=True)
    async with aiohttp.ClientSession(
        connector=connector,
        json_serialize=lambda x: json.dumps(x, ensure_ascii=False)
    ) as session:
        response = await session.post(
            url,
            headers=headers,
            json=payload,
            timeout=aiohttp.ClientTimeout(total=timeout)
        )
        data = await response.json(encoding='utf-8')
        print(f"url={url}, headers={headers}, payload={payload}, timeout={timeout}")
        print(data)
        return data["choices"][0]["message"]["content"]

async def process_request(request_json_fpath) -> str:
    with open(request_json_fpath, 'r') as f:
        request_json = json.load(f)
    url, headers, payload, timeout = request_json['url'], request_json['headers'], request_json['payload'], request_json['timeout']
    
    return await prompt_openrouter(url, headers, payload, timeout)

In [ ]:
await process_request("/home/jz4391/openrouter_proxy/request_test.json")